# Example of generating a 1D2DRR D-HYDRO model - an overview of functionalities

This notebook gives an overview of the functionalities of the D-HyDAMO module, part of the Hydrolib environment.

This notebook is based on previous examples of the python package delft3dfmpy, but now connnected to the Hydrolib-core package, which is used for writing a D-HYDRO model. It contains similar functionalities as delft3dfmpy 2.0.3; input data is expected to be according to HyDAMO DAMO2.2 gpkg-format. The example model used here is based on a part of the Oostrumsche beek in Limburg, added with some fictional dummy data to better illustrate functionalities.

Because of the dummy data and demonstation of certain features, the resulting model is not optimal from a hydrologic point of view.

## Load Python libraries and Hydrolib-core functionality

In [1]:
# Basis
from pathlib import Path
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import contextily as cx
import os
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Missing global # gdal: DRIVER_NAME declaration in C:\Users\dupuits\Miniconda3\Lib\site-packages\osgeo\gdal_array.py
Missing global # gdal: DRIVER_NAME declaration in C:\Users\dupuits\Miniconda3\Lib\site-packages\osgeo\gdal_array.py


In [2]:
## In not installed, add a path from where hydrolib it can be imported
#sys.path.insert(0, "d:/Documents/GitHub/HYDROLIB")
#sys.path.insert(0, r"../../")

# NOTE: core and dhydamo need to be in the same folder to be imported correctly
# and from hydrolib-core
from hydrolib.core.dimr.models import DIMR, FMComponent
from hydrolib.core.dflowfm.inifield.models import IniFieldModel, DiskOnlyFileModel
from hydrolib.core.dflowfm.onedfield.models import OneDFieldModel
from hydrolib.core.dflowfm.structure.models import *
from hydrolib.core.dflowfm.crosssection.models import *
from hydrolib.core.dflowfm.ext.models import ExtModel
from hydrolib.core.dflowfm.mdu.models import FMModel
from hydrolib.core.dflowfm.bc.models import ForcingModel
from hydrolib.core.dflowfm.friction.models import FrictionModel
from hydrolib.core.dflowfm.obs.models import ObservationPointModel

# Importing relevant classes from Hydrolib-dhydamo
from hydrolib.dhydamo.core.hydamo import HyDAMO
from hydrolib.dhydamo.converters.df2hydrolibmodel import Df2HydrolibModel
from hydrolib.dhydamo.geometry import mesh
from hydrolib.dhydamo.core.drr import DRRModel
from hydrolib.dhydamo.core.drtc import DRTCModel
from hydrolib.dhydamo.io.dimrwriter import DIMRWriter
from hydrolib.dhydamo.io.drrwriter import DRRWriter
from hydrolib.dhydamo.geometry.viz import plot_network

Define in- and output paths

In [3]:
# path to the package containing the dummy-data
data_path = Path("../tests/data").resolve()
assert data_path.exists()

# path to write the models
output_path = Path("../tests/model").resolve()
# assert output_path.exists()

Define components that should be used in the model. 1D is used in all cases.

In [4]:
TwoD = True
RR = True
RTC = True

## Read HyDAMO DAMO2.2 data

In [5]:
# all data is contained in one geopackage called 'Example model'
gpkg_file = str(r"D:\Users\dupuits\Desktop\HYDROLIB\dhydamo_testdata_dommel\test_data2_3.gpkg")
# gpkg_file = str(data_path / "Example_model.gpkg")
# initialize a hydamo object
# hydamo = HyDAMO(extent_file=data_path / "Oostrumschebeek_extent.shp")
hydamo = HyDAMO()

# show content
hydamo.branches.show_gpkg(gpkg_file)

Content of gpkg-file D:\Users\dupuits\Desktop\HYDROLIB\dhydamo_testdata_dommel\test_data2_3.gpkg, containing 11 layers:
	INDEX	|	NAME                        	|	GEOM_TYPE      	|	 NFEATURES	|	   NFIELDS
	    0	|	kunstwerkopening              	|	None        	|	      6600	|	        29
	    1	|	hydroobject                   	|	LineString  	|	     31517	|	        46
	    2	|	brug                          	|	Point       	|	      1143	|	        40
	    3	|	duikersifonhevel              	|	LineString  	|	     32409	|	        76
	    4	|	profielpunt                   	|	Point       	|	     64310	|	        19
	    5	|	profiellijn                   	|	LineString  	|	     65320	|	        27
	    6	|	profielgroep                  	|	None        	|	     65320	|	        18
	    7	|	stuw                          	|	Point       	|	      3550	|	        64
	    8	|	regelmiddel                   	|	Point       	|	      3235	|	        43
	    9	|	gemalen                       	|	Point       	|	        71	|

In [6]:
# read branchs
hydamo.branches.read_gpkg_layer(gpkg_file, layer_name="HydroObject", index_col="code")
hydamo.branches

,code,geometry,objectid,se_anno_cad_data,naam,statusobject,hyperlink,opmerking,statusleggerwatersysteem,soortoppwaterkwaliteit,...,nauwkeurigheidxy,nauwkeurigheidz,sde_state_id,created_user,created_date,last_edited_user,last_edited_date,globalid,ws_dommelid,ws_topdesknr
code,,,,,,,,,,,,,,,,,,,,,
BEDO-1095,BEDO-1095,"LINESTRING (154370.70300 409303.89700, 154378....",160000,None,None,3.0,None,None,1.0,NaN,...,None,None,0,None,None,JURRJ1,2021-05-27T06:48:10,{8E3B9BEF-2FCC-4DB1-BF46-ED8EF70D5571},BEDO-1095_HO1,None
OWL60157,OWL60157,"LINESTRING (167970.09200 371340.17700, 167824....",160001,None,None,3.0,None,None,1.0,NaN,...,None,None,0,None,None,STEID1,2023-04-12T11:59:34,{96BA0069-79B7-418B-B93F-C36FCEDCC996},OWL60157_HO1,None
OWL20162,OWL20162,"LINESTRING (153251.81800 408620.46700, 153231....",160002,None,None,3.0,None,None,1.0,NaN,...,None,None,0,None,None,JURRJ1,2022-10-21T05:04:23,{28513C6D-998D-47FC-843D-37BFB1F11761},OWL20162_HO1,None
OWL40221,OWL40221,"LINESTRING (139653.57900 373194.19600, 139683....",160003,None,None,3.0,None,None,1.0,NaN,...,None,None,0,None,None,JURRJ1,2022-10-21T05:04:05,{5BD26884-C2B9-47DB-B6D3-18D6AFB99524},OWL40221_HO01,None
BODO-0192,BODO-0192,"LINESTRING (151208.23700 374084.14200, 151223....",160004,None,None,3.0,None,None,1.0,NaN,...,None,None,0,None,None,JURRJ1,2022-10-21T05:03:26,{B873A955-D6D3-4C22-9027-A2630FDE4E26},BODO-0192_HO1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OWL41978,OWL41978,"LINESTRING (159534.62200 395999.71500, 159513....",161220,None,None,3.0,None,None,1.0,NaN,...,None,None,0,None,None,JURRJ1,2022-10-21T05:04:05,{012452A7-BF38-4F5B-A922-73B62D00F43A},OWL41978_HO1,None
OWL00895,OWL00895,"LINESTRING (139120.56700 397510.75100, 139123....",161221,None,None,3.0,None,None,1.0,NaN,...,None,None,0,None,None,JURRJ1,2021-05-27T07:24:25,{89A60A2F-E973-4334-B628-574512E8A176},OWL00895_HO1,None
OWL21622,OWL21622,"LINESTRING (147845.63710 400630.67290, 147851....",161222,None,None,3.0,None,Objecten voor DSO werkingsgebieden: In overleg...,2.0,NaN,...,None,None,0,None,None,JURRJ1,2021-05-27T07:33:35,{D0AB8930-BBCC-46E5-998E-8781E5DDEB8A},OWL21622_HO1,None


In [7]:
# read profiles
hydamo.profile.read_gpkg_layer(
    gpkg_file,
    layer_name="ProfielPunt",
    groupby_column="profiellijnid",
    order_column="codevolgnummer",
    id_col="code",
    index_col="code"
)
hydamo.profile

Ignoring profiellijnid "{6FF9C890-7414-4E9F-A624-FAC17F45208E}": contains less than two points.
Ignoring profiellijnid "{26698306-139A-47D7-A282-FF71A6CCB9F5}": contains less than two points.
Ignoring profiellijnid "{E936AA8A-FFF8-4E11-8497-0A9103871BE1}": contains less than two points.
Ignoring profiellijnid "{556EC822-2707-4627-8CD3-B297D37BEA74}": contains less than two points.


,code,geometry,globalid,profiellijnid,objectid,se_anno_cad_data,soortmeetpunt,codevolgnummer,richting,tekencode,hoogte,afstand,created_user,created_date,last_edited_user,last_edited_date,ws_dommelid,id,name
code,,,,,,,,,,,,,,,,,,,
None,None,"LINESTRING Z (159930.327 369145.362 25.796, 15...",{7153F3E0-2FB6-4D48-B46B-FDA329CD2B14},{AFE68034-6CA3-4BDF-810F-573626709D6E},4128165,None,98,1,None,NaN,25.80,0.0,FME_PY,2023-09-25T10:54:10,FME_PY,2023-09-25T10:54:10,20230921-DP1_1,0,Keersop
None,None,"LINESTRING Z (155208.241 370379.117 26.032, 15...",{7312C608-198D-4231-8186-699519BD1057},{064687F7-3CCF-44BA-BE56-0ECBF89D8358},4131510,None,98,1,None,NaN,26.03,0.0,FME_PY,2023-07-27T08:48:36,FME_PY,2023-07-27T08:48:36,20230724-DP3_1,0,Keersop
None,None,"LINESTRING Z (155182.029 370296.833 26.028, 15...",{EF112EFD-804E-4E0B-A28A-92F15F4BC7A1},{582703BB-35CF-4A67-A22E-D8D0EA926AF9},4131540,None,98,1,None,NaN,26.03,0.0,FME_PY,2023-07-27T08:48:36,FME_PY,2023-07-27T08:48:36,20230724-DP2_1,0,Keersop
None,None,"LINESTRING Z (155101.767 370298.863 26.008, 15...",{3D7E26B1-93DE-45E0-815B-E05A3CF24203},{4B629ACD-A3D8-42D7-87D9-E50000CF270E},4131503,None,98,1,None,NaN,26.01,0.0,FME_PY,2023-07-27T08:48:36,FME_PY,2023-07-27T08:48:36,20230724-DP1_1,0,Keersop
None,None,"LINESTRING Z (154801.087 370358.358 26.677, 15...",{25C8F589-83C1-4D0A-B8DA-B04E9EF226C2},{CE8AFF5D-E7EE-423D-99F7-7880F2417732},4131536,None,98,1,None,NaN,26.68,0.0,FME_PY,2023-07-27T08:48:36,FME_PY,2023-07-27T08:48:36,20230724-DP7_1,0,Keersop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
None,None,"LINESTRING Z (159861.287 368853.088 27.394, 15...",{F8A87F32-410C-449F-953E-E86FCC808EF4},{0E3D4E64-B991-4DE2-ADCC-1E1E2ABDD1DD},4111094,None,98,1,None,NaN,27.39,0.0,FME_PY,2023-03-07T14:20:53,FME_PY,2023-03-07T14:20:53,DP-ST0001866_1,0,Keersop
None,None,"LINESTRING Z (157561.710 378054.524 18.482, 15...",{C07387DA-6758-4E81-A9F4-8CF8BB21B768},{7676E911-BA0D-4676-954D-1EDA1A62427A},4125728,None,98,1,None,NaN,18.48,0.0,FME_PY,2023-03-21T12:45:02,FME_PY,2023-03-21T12:45:02,DP-ST0001895_1,0,Keersop
None,None,"LINESTRING Z (157562.466 378055.025 18.531, 15...",{4D4C606C-472A-4F26-80D2-84ACCD9209CB},{31EE53F9-2112-4767-BE82-B32790E6E3B0},4125735,None,98,1,None,NaN,18.53,0.0,FME_PY,2023-03-21T12:45:02,FME_PY,2023-03-21T12:45:02,20230320-DP1_1,0,Keersop


In [ ]:
hydamo.profile_roughness.read_gpkg_layer(gpkg_file, layer_name="RuwheidProfiel")
# hydamo.profile.snap_to_branch(hydamo.branches, snap_method="intersecting")
# hydamo.profile.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.profile_roughness

In [ ]:
hydamo.profile_line.read_gpkg_layer(gpkg_file, layer_name="profiellijn")
hydamo.profile_line

In [ ]:
hydamo.profile_group.read_gpkg_layer(gpkg_file, layer_name="profielgroep")
hydamo.profile_group

In [ ]:
hydamo.culverts.read_gpkg_layer(gpkg_file, layer_name="DuikerSifonHevel", index_col="code")
hydamo.culverts

In [ ]:
hydamo.weirs.read_gpkg_layer(gpkg_file, layer_name="Stuw", index_col="code")
hydamo.weirs

In [ ]:
hydamo.opening.read_gpkg_layer(gpkg_file, layer_name="Kunstwerkopening")
hydamo.opening

In [ ]:
hydamo.management_device.read_gpkg_layer(gpkg_file, layer_name="Regelmiddel")
hydamo.management_device

In [ ]:
hydamo.pumpstations.read_gpkg_layer(gpkg_file, layer_name="Gemaal", index_col="code")

In [ ]:
hydamo.pumps.read_gpkg_layer(gpkg_file, layer_name="Pomp", index_col="code")
hydamo.pumps

In [ ]:
hydamo.management.read_gpkg_layer(gpkg_file, layer_name="Sturing", index_col="code")
hydamo.management

In [ ]:
hydamo.bridges.read_gpkg_layer(gpkg_file, layer_name="Brug", index_col="code")
hydamo.bridges

In [ ]:
# read boundaries
hydamo.boundary_conditions.read_gpkg_layer(
    gpkg_file, layer_name="hydrologischerandvoorwaarde", index_col="code"
)
hydamo.boundary_conditions

In [ ]:
# read catchments
hydamo.catchments.read_gpkg_layer(gpkg_file, layer_name="afvoergebiedaanvoergebied", index_col="code")
hydamo.catchments

In [ ]:
hydamo.laterals.read_gpkg_layer(gpkg_file, layer_name="lateraleknoop")
hydamo.laterals